# Установка необходимых зависимостей

In [ ]:
# !pip install -r requirements.txt

# Импорты

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import optuna

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator

# Clssical ML Regression Models
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import (
    LinearRegression,
    Lasso,
    Ridge,
    ElasticNet,
    SGDRegressor,
)
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    BaggingRegressor,
    RandomForestRegressor,
    AdaBoostRegressor,
    GradientBoostingRegressor
)
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings("ignore")

# Загрузка данных

In [ ]:
X = pd.read_csv("X.csv", index_col=0)
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1008 entries, 0 to 1007
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cpm               1008 non-null   float64
 1   hour_start        1008 non-null   int64  
 2   hour_end          1008 non-null   int64  
 3   audience_size     1008 non-null   int64  
 4   duration          1008 non-null   int64  
 5   publishers_count  1008 non-null   int64  
 6   middle_hour       1008 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 63.0 KB


In [ ]:
X.head()

,cpm,hour_start,hour_end,audience_size,duration,publishers_count,middle_hour
0,220.0,1058,1153,1906,95,2,1106
1,312.0,1295,1301,1380,6,2,1298
2,70.0,1229,1249,888,20,6,1239
3,240.0,1295,1377,440,82,2,1336
4,262.0,752,990,1476,238,4,871


In [ ]:
y = pd.read_csv("y.csv", index_col=0)
y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1008 entries, 0 to 1007
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   at_least_one    1008 non-null   float64
 1   at_least_two    1008 non-null   float64
 2   at_least_three  1008 non-null   float64
dtypes: float64(3)
memory usage: 31.5 KB


In [ ]:
y.head()

,at_least_one,at_least_two,at_least_three
0,0.0430,0.0152,0.0073
1,0.0130,0.0000,0.0000
2,0.0878,0.0135,0.0000
3,0.2295,0.1295,0.0727
4,0.3963,0.2785,0.2270


# Разбиение выборки и стандаризация

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((806, 7), (202, 7), (806, 3), (202, 3))

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
train_data = (X_train_scaled, y_train)
test_data = (X_test_scaled, y_test)

# Целевая метрика заказчика

In [ ]:
EPS = 0.005

def log_mape_column_value(responses_column, answers_column, epsilon=EPS):
    return np.abs(np.log(
        (responses_column + epsilon) / (answers_column + epsilon)
    )).mean()

def mean_log_accuracy_ratio(answers, responses, epsilon=EPS):
    log_accuracy_ratio_mean = np.array(
        [
            log_mape_column_value(responses.at_least_one, answers.at_least_one, epsilon),
            log_mape_column_value(responses.at_least_two, answers.at_least_two, epsilon),
            log_mape_column_value(responses.at_least_three, answers.at_least_three, epsilon),
        ]
    ).mean()

    percentage_error = 100 * (np.exp(log_accuracy_ratio_mean) - 1)

    return percentage_error.round(decimals=2)

# Скрипты автоматизации эксперементальной апробации моделей

In [ ]:
def try_model_on_sample(
    model_class: BaseEstimator,
    train_data: tuple[pd.DataFrame],
    test_data: tuple[pd.DataFrame],
    **model_params
) -> float:

    y_pred = test_data[1].copy()

    for i in range(3):
        model = model_class(**model_params)
        model.fit(train_data[0], train_data[1].iloc[:, i])
        pred = model.predict(test_data[0])
        y_pred.iloc[:, i] = pred

    return mean_log_accuracy_ratio(y_pred, test_data[1])


def try_model(
    model_class: BaseEstimator,
    X: pd.DataFrame,
    y: pd.DataFrame,
    n_times: int=5,
    **model_params
) -> float:

    results = np.zeros(n_times)

    for i in range(n_times):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y,
            test_size=0.2
        )

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        train_data = (X_train_scaled, y_train)
        test_data = (X_test_scaled, y_test)

        results[i] = try_model_on_sample(
            model_class,
            train_data,
            test_data,
            **model_params
        )

    return results.mean()

In [ ]:
# Пустой pd.Series для сохранения результатов
model_results = pd.Series()

# Случайное предсказание

In [ ]:
dummy_result = try_model(
    DummyRegressor, X, y
)

model_results["dummy"] = dummy_result
dummy_result

375.37800000000004

# Линейные модели

In [ ]:
linear_regression_result = try_model(
    LinearRegression, X, y
)

model_results["linear_regression"] = linear_regression_result
linear_regression_result

184.33800000000002

In [ ]:
lasso_regression_result = try_model(
    Lasso, X, y
)

model_results["lasso"] = lasso_regression_result
lasso_regression_result

345.53600000000006

In [ ]:
ridge_regression_result = try_model(
    Ridge, X, y
)

model_results["ridge"] = ridge_regression_result
ridge_regression_result

188.53

In [ ]:
elastic_net_regression_result = try_model(
    ElasticNet, X, y
)

model_results["elastic_net"] = elastic_net_regression_result
elastic_net_regression_result

357.204

In [ ]:
sgd_regression_result = try_model(
    SGDRegressor, X, y
)

model_results["sgd"] = sgd_regression_result
sgd_regression_result

187.506

# Метод опорных веторов

In [ ]:
svr_regression_result = try_model(
    SVR, X, y
)

model_results["svm"] = svr_regression_result
svr_regression_result

327.18600000000004

# Дерево решений

In [ ]:
tree_regression_result = try_model(
    DecisionTreeRegressor, X, y
)

model_results["tree"] = tree_regression_result
tree_regression_result

132.552

# Ансаблевые беггинговые методы

In [ ]:
bagging_regression_result = try_model(
    BaggingRegressor, X, y
)

model_results["bagging"] = bagging_regression_result
bagging_regression_result

100.298

In [ ]:
random_forest_regression_result = try_model(
    RandomForestRegressor, X, y
)

model_results["random_forest"] = random_forest_regression_result
random_forest_regression_result

99.232

# Ансаблевые бустинговые методы

In [ ]:
ada_boost_regression_result = try_model(
    AdaBoostRegressor, X, y
)

model_results["ada_boost"] = ada_boost_regression_result
ada_boost_regression_result

297.878

In [ ]:
gradient_boost_regression_result = try_model(
    GradientBoostingRegressor, X, y
)

model_results["gradient_boost"] = gradient_boost_regression_result
gradient_boost_regression_result

146.846

In [ ]:
xgb_regression_result = try_model(
    XGBRegressor, X, y
)

model_results["xgb"] = xgb_regression_result
xgb_regression_result

136.90800000000002

In [ ]:
cat_boost_regression_result = try_model(
    CatBoostRegressor, X, y, silent=True
)

model_results["cat_boost"] = cat_boost_regression_result
cat_boost_regression_result

138.642

# Сравнение результатов

In [ ]:
model_results.sort_values()

,0
random_forest,99.232
bagging,100.298
tree,132.552
xgb,136.908
cat_boost,138.642
gradient_boost,146.846
linear_regression,184.338
sgd,187.506
ridge,188.530
ada_boost,297.878


На основе представленных результатов, `беггинговые` методы (в частности `Random Fores0t`) являются наиболее эффективными моделями для данной задачи, вероятно это вызвано тем, что будстрапирование лучше подходит в условиях довольно малой выборки. Модели, такие как `XGBoost` и `CatBoost`, также показывают хорошие результаты и могут быть полезны в зависимости от требований к интерпретируемости. В то же время, линейные модели и прочие методы, такие как `SVM` и `AdaBoost`, не показали хороших результатов.

# Оптимизация гипер-параметров

In [ ]:
def optimize_rf_hyperparameters(
    X, y
) -> dict:

    def objective(trial):

        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2
        )

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        # Определяем гиперпараметры для оптимизации
        n_estimators = trial.suggest_int('n_estimators', 10, 300)
        max_depth = trial.suggest_int('max_depth', 1, 50)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])
        max_features = trial.suggest_int('max_features', 2, 7)

        y_pred = y_val.copy()

        for target in y_train.columns:
            # Создаем модель Random Forest с заданными гиперпараметрами
            model = RandomForestRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                bootstrap=bootstrap,
                max_features=max_features
            )

            model.fit(X_train, y_train[target])
            y_pred[target] = model.predict(X_val)

        return mean_log_accuracy_ratio(y_val, y_pred)

    # Создаем объект исследования
    study = optuna.create_study(direction='minimize')  # Минимизация оценки
    study.optimize(objective, n_trials=100, n_jobs=-1)  # Оптимизация на 100 испытаниях

    # Возвращаем наилучшие гиперпараметры
    return study.best_params

In [ ]:
best_rf_params = optimize_rf_hyperparameters(
    X_train, y_train
)

[I 2024-12-16 18:31:49,759] A new study created in memory with name: no-name-aaed875c-8765-40ba-8477-3687d35bfac0
[I 2024-12-16 18:31:49,999] Trial 0 finished with value: 117.06 and parameters: {'n_estimators': 20, 'max_depth': 44, 'min_samples_split': 5, 'min_samples_leaf': 5, 'bootstrap': False, 'max_features': 2}. Best is trial 0 with value: 117.06.
[I 2024-12-16 18:31:50,935] Trial 2 finished with value: 96.65 and parameters: {'n_estimators': 53, 'max_depth': 22, 'min_samples_split': 10, 'min_samples_leaf': 1, 'bootstrap': False, 'max_features': 6}. Best is trial 2 with value: 96.65.
[I 2024-12-16 18:31:51,187] Trial 1 finished with value: 89.84 and parameters: {'n_estimators': 110, 'max_depth': 21, 'min_samples_split': 7, 'min_samples_leaf': 4, 'bootstrap': False, 'max_features': 4}. Best is trial 1 with value: 89.84.
[I 2024-12-16 18:31:52,095] Trial 3 finished with value: 109.85 and parameters: {'n_estimators': 105, 'max_depth': 18, 'min_samples_split': 10, 'min_samples_leaf': 5

In [ ]:
best_rf_params

{'n_estimators': 155,
 'max_depth': 28,
 'min_samples_split': 8,
 'min_samples_leaf': 9,
 'bootstrap': False,
 'max_features': 4}

# Результаты

In [ ]:
try_model(
    RandomForestRegressor,
    X, y, **best_rf_params
)

98.074

Метрику удалось улучшиить в среднем, но не значительно. Однако в условиях сильной ограниченности размера выборки это можно считать приемлимым результатом.

# Feature imporance

In [ ]:
feature_imp_array = regressors["at_least_one"].feature_importances_
feature_imp = pd.Series({
    col: imp for col, imp in zip(X_train.columns, feature_imp_array)
})

feature_imp.sort_values(ascending=False)

,0
duration,0.495034
cpm,0.158380
audience_size,0.087140
middle_hour,0.073053
hour_start,0.067366
publishers_count,0.065330
hour_end,0.053697


1. **Наиболее важный признак**:
   - **Duration (длительность)** имеет наивысшую важность. Это может означать, что длительность является ключевым фактором в модели и, вероятно, играет центральную роль в прогнозировании.

2. **Средняя важность**:
   - **CPM (стоимость за тысячу показов)** и **Audience Size (размер аудитории)** также имеют заметную важность, хотя и значительно ниже, чем у длительности. Это говорит о том, что эти факторы также влияют на модель, но не так сильно, как длительность.

3. **Меньшая важность**:
   - **Middle Hour (средний час)**, **Hour Start (начальный час)**, **Publishers Count (количество издателей)** и **Hour End (конечный час)** имеют более низкие значения важности. Эти признаки могут иметь некоторую степень влияния на модель, но, судя по их значению, они менее значимы по сравнению с длительностью и CPM.

Важно отметить, что важность признаков не обязательно указывает на причинно-следственную связь. Высокая важность не всегда означает, что признак является причиной изменения целевой переменной; это может быть связано с корреляцией с другими признаками.

Учитывая, что **duration** является самым важным признаком, стоит обратить внимание на его точность и качество данных. Возможно, стоит провести дополнительный анализ, чтобы понять, как именно он влияет на целевую переменную.

Признаки с низкой важностью могут быть рассмотрены для исключения из модели, если они не добавляют значительной информации, что может помочь упростить модель и улучшить ее интерпретируемость, но учитывая малое количество признаков, это имеет смысл только при добавлении других более информативных признаков.